## 管理层信息抽取

In [ ]:
! cp /Users/alchemy_taotaox/Desktop/next/my_test/* ../corpus/

In [ ]:
! ls ../corpus

## 算法基本思路
1. 扫描pdf 获得页眉页脚
2. 扫描pdf 获得最小和最大的x坐标
    如果一个字符是最小的坐标，那么就和之前的行合并为一个段落
    如果一个字符是最大的坐标，那么就和之前的
2. 提取文档中的表格和字符串。

In [ ]:
from pdfplumber.pdf import PDF
import re
from collections import Counter

def is_header(str, header):
    if header is None:
        return False
    return str == header

def is_foot(str, foot):
    if foot is None:
        return False
    
    s = re.sub(r'\d+', '#num#', str)
    return s == foot

def get_foot_header(pdf:PDF, check_pages_num=20, threshold = 0.9):
    head_counter = Counter()
    foot_counter = Counter()
    pages_num = len(pdf.pages)
    if pages_num < check_pages_num:
        check_pages_num = pages_num
    for i in range(check_pages_num):
        text_lines = pdf.pages[i].extract_words()
        top_line = text_lines[0]['text']
        bottom_line = text_lines[-1]['text']
        bottom_line = re.sub(r'\d+', '#num#', bottom_line)
        head_counter.update([top_line])
        foot_counter.update([bottom_line])
        
    most_common_head_word, count_head = head_counter.most_common(1)[0]
    most_common_foot_word, count_foot = foot_counter.most_common(1)[0]
    head, foot = None, None
    if count_head *1./check_pages_num >= threshold:
        head = most_common_head_word
    if count_foot *1./check_pages_num >= threshold:
        foot = most_common_foot_word
    return head, foot
    
    

In [ ]:
import pdfplumber



In [81]:
from operator import itemgetter

def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]



def get_lines_from_page(page, head, foot):
    tables = page.find_tables()
    table_bboxes = [i.bbox for i in tables]
    tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
    non_table_words = [word for word in page.extract_words() if not any(
        [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
    lines = []
    lines_x0_x1 =[]
    for cluster in pdfplumber.utils.cluster_objects(
        non_table_words + tables, itemgetter('top'), tolerance=5):
        if 'text' in cluster[0]:
            x_0 = x_1 = -1
            inner_lines = []
            for item in cluster:
                if len(cluster) == 1 and (is_foot(item['text'], foot) or  is_header(item['text'], head)):
                    continue
                if x_0 == -1:
                    x_0 = item['x0']
                x_1 = item['x1']
                inner_lines.append(item['text'])
            if len(inner_lines) > 0:
                lines.append(' '.join(inner_lines))
                lines_x0_x1.append((x_0, x_1))
                
            # lines.append(' '.join([i['text'] for i in cluster if not is_foot(i['text'], foot) and not is_header(i['text'], head)]))
        elif 'table' in cluster[0]:
            lines.append(cluster[0]['table'])
            lines_x0_x1.append((10000000, -1))

    # Find the minimum of the first elements
    if len(lines_x0_x1) == 0:
        return []
    min_first = min(x[0] for x in lines_x0_x1)

    # Find the maximum of the second elements
    max_second = max(x[1] for x in lines_x0_x1)
       
    # print(lines)

    for index, item in enumerate(lines):
        if type(lines[index]) == str:
            lines[index] = lines[index].strip()
        # 如果锁进， 那么要换行
        
        if abs(lines_x0_x1[index][0] - min_first) > 0.1 and lines_x0_x1[index][0] < 1000000:
            # print(lines_x0_x1[index][0], min_first, "----")
            lines[index] = "\n" + lines[index]
        # 如果没有写完，那么换行
        if abs(lines_x0_x1[index][1] - max_second) > 16 and lines_x0_x1[index][1] > 0:
            # print(lines_x0_x1[index][1], max_second, "----", lines[index])
            lines[index] = lines[index] + "\n"
        elif lines_x0_x1[index][1] > 0:
            # print('******FULL LINE******', lines_x0_x1[index][1], max_second, "----", lines[index])
            pass
        # print(lines[index], end='')
    return lines

In [82]:
filename = "../corpus/year_report_2.pdf"
pdf = pdfplumber.open(filename)
page = pdf.pages[10]
page = pdf.pages[15]
page = pdf.pages[10]
# print(page.extract_text())
head, foot = get_foot_header(pdf)
print(head)
print(foot)
get_lines_from_page(page, head, foot)

print(pdf.pages[10].extract_words() )# 计算一个字符大概的宽度是多少，然后定义magic distance = 16
print(pdf.pages[10].extract_text())

上海安诺其集团股份有限公司2022年年度报告全文
#num#
[{'text': '上海安诺其集团股份有限公司2022年年度报告全文', 'x0': 333.72, 'x1': 538.75, 'top': 43.545920000000024, 'doctop': 8462.74592, 'bottom': 52.66592000000003, 'upright': True, 'direction': 1}, {'text': '年，受国际动荡局势影响，全球大宗商品价格持续高位波动，印染行业在成本端承受较大压力。国家统计局数据显示，', 'x0': 56.664, 'x1': 529.2872800000001, 'top': 79.30592000000001, 'doctop': 8498.50592, 'bottom': 88.42592000000002, 'upright': True, 'direction': 1}, {'text': '2022', 'x0': 56.664, 'x1': 76.85477999999999, 'top': 103.18783999999994, 'doctop': 8522.38784, 'bottom': 112.30783999999994, 'upright': True, 'direction': 1}, {'text': '年我国能源价格上涨', 'x0': 80.904, 'x1': 162.27264, 'top': 102.58591999999999, 'doctop': 8521.785919999998, 'bottom': 111.70591999999999, 'upright': True, 'direction': 1}, {'text': '11.2%，原材料价格上涨', 'x0': 166.63, 'x1': 264.55863999999997, 'top': 102.58591999999999, 'doctop': 8521.785919999998, 'bottom': 112.30783999999994, 'upright': True, 'direction': 1}, {'text': '10.3%，生产资料价格大幅上涨叠加内需消费不足，行业利润空间', 'x0'

In [87]:
from pdfplumber.pdf import PDF
import json
def begin_mda(lines):
    if len(lines) < 1:
        return False
    if type(lines[0]) != str:
        return False
    return lines[0].find('管理层讨论与分析') != -1
def end_mda(lines):
    if len(lines) < 1:
        return False
    if type(lines[0]) != str:
        return False
    # print("check: ", lines[0], "$$$$$$")
    return lines[0].find('公司治理') != -1

def get_all_lines_about_mda(pdf:PDF, head:str, foot:str):
    begin = False
    end = False
    all_lines = []
    for page in pdf.pages:
        lines = get_lines_from_page(page, head, foot)
        if begin is False and begin_mda(lines):
            begin = True
        if begin and end_mda(lines):
            break
        if begin is True:
            all_lines.append(lines)

    content_list = []
    for lines in all_lines:
        for line in lines:
            if type(line) != str:
                line = json.dumps(line, ensure_ascii=False)
            # print(line, end='')
            content_list.append(line)
    return "".join(content_list)
            
content = get_all_lines_about_mda(pdf, head, foot)
print(content)


第三节 管理层讨论与分析
一、报告期内公司所处行业情况
公司需遵守《深圳证券交易所上市公司自律监管指引第3号——行业信息披露》中的“化工行业相关业务”的披露要求
化工基本上可以分为石油化工、基础化工和精细化工三大类，我国把产量小、按不同化学结构进行生产和销售的化学物质，及经过加工配制、具有专门功能或最终使用性能的产品统一称为精细化学品。精细化工产品下游应用领域广阔，其中新材料、服装纺织、电子等领域为下游应用领域。

我国精细化工产业发展情况：精细化工是化学工业发展的战略重点，目前，我国在精细化学品工业具有较强的竞争力，是世界重要精细化学品生产基地。根据国家统计局数据，2009年底，我国化学原料及化学制品制造业规模以上企业共28,120家，当年实现主营业务收入36,297.99亿元；至2017年底，全国化学原料及化学制品制造业企业为24,869
家，当年实现主营业务收入81,889.06亿元，年均复合增长率为12.33%，集中化、规模化进程较为显著。

（一）报告期内，染料行业运行情况

2022年我国染颜料行业呈低迷状态，染颜料、中间体、印染助剂等行业经济运行指标总体较 2021年同期均有下降。2022年染颜料产量合计完成109.7万吨，同比下降2.1%；全年染颜料工业总产值完成664.5亿元，同比下降4.9%；销售收入累计完成618.9亿元，同比下降 3.5%；利税总额完成68.4亿元，同比下降28.3%；其中染料完成83.5万吨，同比下降2.3%；有机颜料完成26.2万吨，同比下降1.1%；染颜料中间体完成49.5万吨，同比下降4.1%。（以上内容来源于染料行业协会，上述数据中包含颜料、助剂、中间体行业数据）

（二）报告期内，下游印染行业经济运行情况

2022年，国际环境仍然复杂严峻，地缘政治冲突不断深化，发达经济货币政策持续收紧，全球经济下行风险加大，外需收缩进一步显现。国内消费市场需求下降对印染企业正常生产经营造成较大影响。印染行业发展环境的复杂性、严峻性、不确定性显著上升。全年来看，印染行业生产、效益指标呈现波动下滑态势，2022年行业企业面临的生产经营压力明显增大。

根据国家统计局数据，2022年 1-12月，印染行业规模以上企业印染布产量 556.22亿米，同比下降 7.52%。2022年，受国际动荡局势影响，全球大宗商品价格持续高位波动，印染